In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os,sys
sys.path.append('/content/drive/MyDrive/iapr/project/')
import cv2 as cv
import matplotlib.pyplot as plt
import PIL.Image
import numpy as np
from typing import Union
from glob import glob
import pandas as pd
import torchvision.models as models
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms
import time


Mounted at /content/drive


In [6]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE=32
print('CUDA INDEX: {}'.format(DEVICE.index))

CUDA INDEX: 0


## Dataset

In [39]:
# class Iapr(Dataset):
#     def __init__(self, ims_obj, labels, transform=None):
#         self.transform = transform
#         self.labels=labels
#         self.ims_obj=np.array(ims_obj)

#     def __getitem__(self, index):
#         img = self.ims_obj[index]
#         if self.transform is not None:
#           img = self.transform(img)
#         label = self.labels[index]

#         return img,label

#     def __len__(self):
#         return self.ims_obj.shape[0]

class Iapr(Dataset):
    def __init__(self, ims_obj, labels, transform=None):
        self.transform = transform

    def __getitem__(self, index):
        img = img_obj[index] #to convert grayscale images to RGB.

        if self.transform is not None:
          img = self.transform(img)
        
        label = labels[index]
        return img, label

    def __len__(self):
        return labels.shape[0]

In [20]:
#Transformers
custom_transform = transforms.Compose([#transforms.Lambda(lambda x: x/255.),# not necessary
                                       transforms.Resize((224, 224)),
                                       #transforms.RandomCrop((224, 224)),
                                       #transforms.ColorJitter(brightness=0.5),
                                       #transforms.RandomRotation(degrees=45),
                                       #transforms.RandomHorizontalFlip(p=0.1),
                                       #transforms.RandomVerticalFlip(p=0.5),
                                       #transforms.RandomGrayscale(p=0.05),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.0, 0.0, 0.0), (1.0, 1.0, 1.0))
                                       
                                      ])

test_transform = transforms.Compose([#transforms.Lambda(lambda x: x/255.),# not necessary
                                       transforms.Resize((224, 224)),
                                       #transforms.RandomCrop((224, 224)),
                                       #transforms.ColorJitter(brightness=0.5),
                                       #transforms.RandomRotation(degrees=45),
                                       #transforms.RandomHorizontalFlip(p=0.1),
                                       #transforms.RandomVerticalFlip(p=0.5),
                                       #transforms.RandomGrayscale(p=0.05),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.0, 0.0, 0.0), (1.0, 1.0, 1.0))
                                      ])

## Model

In [22]:
class AlexNet(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3, 256, kernel_size=11, stride=4, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            #
            torch.nn.Conv2d(256, 192, kernel_size=5, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            #
            torch.nn.Conv2d(192, 384, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            #
            torch.nn.Conv2d(384, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            #
            torch.nn.Conv2d(256, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = torch.nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256 * 6 * 6, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(4096, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        logits = self.classifier(x)
        return logits

## Train

In [12]:
def compute_accuracy(model, data_loader):
  correct_pred, num_examples = 0, 0
  for i, (features, targets) in enumerate(data_loader):            
      features = features.to(DEVICE)
      targets = targets.to(DEVICE)
      logits = model(features)
      _, predicted_labels = torch.max(logits, 1)
      num_examples += targets.size(0)
      correct_pred += (predicted_labels == targets).sum()
  return correct_pred.float()/num_examples * 100


In [13]:
def train_model(model, num_epochs, train_loader,
                valid_loader, test_loader, optimizer, device):
  start_time = time.time()
  minibatch_loss_list, train_acc_list, valid_acc_list = [], [], []
  for epoch in range(num_epochs):

      model.train()
      for batch_idx, (features, targets) in enumerate(train_loader):

          features = features.to(device)
          targets = targets.to(device, dtype=torch.int64)

          # ## FORWARD AND BACK PROP
          logits = model(features)
          loss = torch.nn.functional.cross_entropy(logits, targets)
          optimizer.zero_grad()

          loss.backward()

          # ## UPDATE MODEL PARAMETERS
          optimizer.step()

          # ## LOGGING
          minibatch_loss_list.append(loss.item())
          # if batch_idx % 50:
          #     print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
          #           f'| Batch {batch_idx+1:04d}/{len(train_loader):04d} '
          #           f'| Loss: {loss:.4f}')
#################################################################################################################################
      model.eval()
      with torch.no_grad():  # save memory during inference
          train_acc = compute_accuracy(model, train_loader#, device=device
                                        )
          valid_acc = compute_accuracy(model, valid_loader#, device=device
                                        )
          print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                f'| Train: {train_acc :.2f}% '
                f'| Validation: {valid_acc :.2f}%')
          train_acc_list.append(train_acc.item())
          valid_acc_list.append(valid_acc.item())

      elapsed = (time.time() - start_time)/60
      print(f'Time elapsed: {elapsed:.2f} min')

  elapsed = (time.time() - start_time)/60
  print(f'Total Training Time: {elapsed:.2f} min')

  #test_acc = compute_accuracy(model, test_loader#, device=device)
  #print(f'Test accuracy {test_acc :.2f}%')

  return minibatch_loss_list, train_acc_list, valid_acc_list

## Pipeline

In [27]:
# dictonary from labels(str) to int
full_dict_iapr={
  'AS': 0, '2S':1, '3S':2, '4S':3, '5S':4, '6S':5, '7S':6,
  '8S':7, '9S':8, '10S':9, 'JS':10, 'QS':11, 'KS':12,
  'AH': 13, '2H': 14, '3H': 15, '4H': 16, '5H': 17, '6H': 18,
  '7H':19, '8H':20, '9H':21, '10H':22, 'JH': 23, 'QH': 24,
  'KH':25,
  'AC': 26, '2C': 27, '3C': 28, '4C': 29, '5C': 30, '6C': 31,
  '7C':32, '8C':33, '9C':34, '10C':35, 'JC': 36, 'QC': 37,
  'KC':38,
  'AD': 39, '2D': 40, '3D': 41, '4D': 42, '5D': 43, '6D': 44,
  '7D':45, '8D':46, '9D':47, '10D':48, 'JD': 49, 'QD': 50,
  'KD':51}

**Raw data link**

  [Label: updated_train_labels.csv](https://drive.google.com/file/d/1L8TglWAo9wKQNXJvVbYkbQIa3wJOLEEO/view?usp=sharing)

  [Data: Tcards.npy](https://drive.google.com/file/d/1eVEAf7qZaTpfqygPjPsCIT5nNrsKcMk7/view?usp=sharing)

In [40]:
# load raw data
import pandas as pd
path = '/content/drive/MyDrive/iapr/project/data/train'
df = pd.read_csv(os.path.join(path,'updated_train_labels.csv')) #filelink: https://drive.google.com/file/d/1eVEAf7qZaTpfqygPjPsCIT5nNrsKcMk7/view?usp=sharing

labels=[]
for i in range(28):
  for t in ['T1','T2','T3','T4','T5']:
    tmp = df.iloc[i]
    labels.append(full_dict_iapr[tmp[t]])
labels = np.array(labels)

import PIL.Image as Image
a=transforms.Resize([224, 224])
arr=np.load("/content/drive/MyDrive/iapr/project/data/Tcards.npy") # 140,348,228,3 Num,H,W,C
img_obj = [Image.fromarray(im.astype(np.uint8)) for im in arr]
print('img_obj len:{}'.format(len(img_obj)))


img_obj len:140


In [41]:
# construct dataset
train_dataset = Iapr(img_obj[0:120],labels,    transform=custom_transform)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          drop_last=True,
                          shuffle=True, # want to shuffle the dataset
                          num_workers=2) # number processes/CPUs to use
valid_dataset = Iapr(
    img_obj[120:130],
    labels,
    transform=test_transform)

valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=12,
    shuffle=False,
    num_workers=2)

test_dataset = Iapr(
    img_obj[-10],
    labels,
    transform=test_transform)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2)


**Pretrained model from Kaggele**

[Model.pt](https://drive.google.com/file/d/1Ehoyi5n9oYqw1oLB_yVEDASeZMW9Z66F/view?usp=sharing)

In [42]:
# Load pre-trained model
path='/content/drive/MyDrive/iapr/project/data_kaggle'
model=AlexNet(num_classes=52)
model = model.to(DEVICE)
model.load_state_dict(torch.load(path+'/model.pt'))
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 256, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(256, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)

### Fine-tune

In [ ]:
#Train and save
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.1)

minibatch_loss_list, train_acc_list, valid_acc_list = train_model(
    model=model,
    num_epochs=50,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    device=DEVICE, 
    )

time=time.strftime('%Y-%m-%d-%H-%M', time.localtime())
torch.save(model.state_dict(), '/content/drive/MyDrive/iapr/project/'+'/model/'+time+'.pt')

### Use fine-tuned model

**Fine-tuned Model**

[Model.pt](https://drive.google.com/file/d/14DJ4M4eEOAncIsyhzzTmh6MqTyFFp5PE/view?usp=sharing)

In [48]:
model=AlexNet(num_classes=52)
model = model.to(DEVICE)
model.load_state_dict(torch.load(path+'/model.pt'))
model.eval()
test_acc = compute_accuracy(model, test_loader)

In [47]:
test_acc

tensor(82.8571, device='cuda:0')